<a href="https://colab.research.google.com/github/tanvi-ap/ClimateChange_StanceDetection/blob/main/climatechangestance_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. [COVID-19 Vaccine — What’s the Public Sentiment](https://towardsdatascience.com/covid-19-vaccine-whats-the-public-sentiment-7149c9b42b99)
2. [COVID-19 Sentiment Analysis Series](https://towardsdatascience.com/covid-19-sentiment-analysis-series-part-1-getting-started-70892122630c)

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
!pip install python-dotenv

In [ ]:
!pip install tweet-preprocessor

In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 5.5 MB/s 
     |████████████████████████████████| 170 kB 41.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=6bd3a0933fe6949b4c6046ff41c7c51273c503329b1f6cb82d320054c3735876
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
!pip install pycorenlp

  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2143 sha256=a51c87962da42ad66c55560e90f86c61d19eaaa8ce6c8cf587ccb3a72d202a95
  Stored in directory: /root/.cache/pip/wheels/83/d8/ad/6b2276343ac605ee47e6beddb28331e96377909e5c816539c3
Successfully built pycorenlp


In [ ]:
!pip install flair

     |████████████████████████████████| 322 kB 5.4 MB/s 
     |████████████████████████████████| 19.7 MB 6.2 MB/s 
     |████████████████████████████████| 48 kB 5.2 MB/s 
     |████████████████████████████████| 788 kB 27.8 MB/s 
     |████████████████████████████████| 61 kB 502 kB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.1 MB 32.4 MB/s 
     |████████████████████████████████| 981 kB 45.7 MB/s 
     |████████████████████████████████| 62 kB 808 kB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=8e2e0b65629abe6c3a44409e67fe29665618571fa9c351

In [ ]:
# General dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import os
import time
from datetime import datetime, date, timedelta

# For Twitter API extraction
import tweepy
#!pip install python-dotenv
from dotenv import load_dotenv

# Tweet pre-processor
import preprocessor as p

# NLTK
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# TextBlob
from textblob import TextBlob

# FLairNLP
from flair.models import TextClassifier
from flair.data import Sentence

# Stanza
import stanza
stanza.download('en')

# Stanford CoreNLP
from pycorenlp import StanfordCoreNLP

import json

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


2021-12-04 15:30:25 INFO: Downloading default packages for language: en (English)...


2021-12-04 15:30:39 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:

f= open('/drive/MyDrive/keys.json','r')
data = json.load(f)

#print(data[0])

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(data[0]['consumer_key'], data[0]['consumer_secret'])
auth.set_access_token(data[0]['access_token'], data[0]['access_secret'])
api = tweepy.API(auth,wait_on_rate_limit=True)

class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 1000:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)  

In [ ]:
# import json

# l = MyStreamListener()
# stream = tweepy.Stream(auth, l)

# stream.filter(track=['covid19 vaccine', 'covid-19 vaccine', 'coronavirus vaccine', 'covid19 vaccines', 'covid-19 vaccines', 'coronavirus vaccines', 'vaccine', 'vaccines'])

# # String of path to file: tweets_data_path
# tweets_data_path = 'tweets.txt'

# # Initialize empty list to store tweets: tweets_data
# tweets_data = []

# # Open connection to file
# tweets_file = open(tweets_data_path, "r")

# # Read in tweets and store in list: tweets_data
# for line in tweets_file:
#     tweet = json.loads(line)
#     tweets_data.append(tweet)

# # Close connection to file
# tweets_file.close()

# # Print the keys of the first tweet dict
# print(tweets_data[0].keys())

In [ ]:
# import pandas as pd

# df = pd.DataFrame(tweets_data, columns=["text","lang"])
# print(df)

                                                  text lang
0    RT @TheInsiderPaper: JUST IN - Johnson &amp; J...   en
1    RT @McGiff: If it looks like bullshit, smells ...   en
2    RT @bloodcancer_uk: This is welcome news from ...   en
3    @WHO @mvankerkhove Could the CCP have variants...   en
4    RT @joshtpm: 4/ them is another story. It does...   en
..                                                 ...  ...
995  RT @WalshFreedom: Bullshit. Biden never lied a...   en
996  RT @NotDevinsMom: The stock market is plummeti...   en
997  RT @ProfessorFerg1: So, apologies for applying...   en
998  RT @Denali_2: Major questions remain about the...   en
999  @WSJ @WSJopinion Arsonist blames fire departme...   en

[1000 rows x 2 columns]


In [ ]:
# Generate list of dates (7 days window) based on today's date
list_of_dates = []
today = date.today()
for i in range(-7,1):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [ ]:
list_of_dicts = []
search_term = 'climate+change global+warming'
num_tweets = 16000

In [ ]:
#https://github.com/kennethleungty/COVID19-Vaccine-Sentiment-Analysis/blob/main/COVID-19%20Vaccine%20Sentiment%20Analysis.ipynb
def get_tweets(search_term = search_term, num_tweets = num_tweets):
    
    for end_date in list_of_dates:
        start_date = '2018-11-01'
        tweet_count = len(list_of_dicts)

        for tweet in tweepy.Cursor(api.search,
                                   q=f'{search_term} since:{start_date} until:{end_date}',
                                   lang = 'en',
                                   count = num_tweets,
                                   tweet_mode = 'extended').items(num_tweets):
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if tweet.lang == "en":
                    tweet_dict = {}
                    tweet_dict['username'] = tweet.user.name
                    tweet_dict['location'] = tweet.user.location
                    tweet_dict['text'] = tweet.full_text
                    #tweet_dict['fav_count'] = tweet.favorite_count  
                    tweet_dict['hashtags'] = tweet.entities['hashtags']
                    tweet_dict['tweet_date'] = tweet.created_at
                    list_of_dicts.append(tweet_dict)
                    tweet_count +=1
                    print(f'Extracted tweet count = {tweet_count}')
                
        print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
        time.sleep(900)
        print('Ready to go again')

In [ ]:
get_tweets()

Extracted tweet count = 1
Extracted tweet count = 2
Extracted tweet count = 3
Extracted tweet count = 4
Extracted tweet count = 5
Extracted tweet count = 6
Extracted tweet count = 7
Extracted tweet count = 8
Extracted tweet count = 9
Extracted tweet count = 10
Extracted tweet count = 11
Extracted tweet count = 12
Extracted tweet count = 13
Extracted tweet count = 14
Extracted tweet count = 15
Extracted tweet count = 16
Extracted tweet count = 17
Extracted tweet count = 18
Extracted tweet count = 19
Extracted tweet count = 20
Extracted tweet count = 21
Extracted tweet count = 22
Extracted tweet count = 23
Extracted tweet count = 24
Extracted tweet count = 25
Extracted tweet count = 26
Extracted tweet count = 27
Extracted tweet count = 28
Extracted tweet count = 29
Extracted tweet count = 30
Extracted tweet count = 31
Extracted tweet count = 32
Extracted tweet count = 33
Extracted tweet count = 34
Extracted tweet count = 35
Extracted tweet count = 36
Extracted tweet count = 37
Extracted 

In [ ]:
 len(list_of_dicts)

8155

In [ ]:
# Transform list of dictionaries into a Pandas dataframe
tweets_df = pd.DataFrame(list_of_dicts)
tweets_df.sort_values(by='tweet_date').reset_index(drop = True)

,username,location,text,hashtags,tweet_date
0,TallyLassie47@,"Tallahassee, FL",@PattyArquette Me too &amp; so many ppl don’t ...,[],2021-11-25 03:09:24
1,TallyLassie47@,"Tallahassee, FL",@PattyArquette Me too &amp; so many ppl don’t ...,[],2021-11-25 03:09:24
2,TallyLassie47@,"Tallahassee, FL",@PattyArquette Me too &amp; so many ppl don’t ...,[],2021-11-25 03:09:24
3,TallyLassie47@,"Tallahassee, FL",@PattyArquette Me too &amp; so many ppl don’t ...,[],2021-11-25 03:09:24
4,TallyLassie47@,"Tallahassee, FL",@PattyArquette Me too &amp; so many ppl don’t ...,[],2021-11-25 03:09:24
...,...,...,...,...,...
7020,Just call it what it is GLOBAL WARMING,"California, USA","@Marciofs_ Nope, thanks to greedy industrialis...",[],2021-12-02 22:58:15
7021,allansharpe,Peterborough,Human Activities Are Making Some Extreme Event...,[],2021-12-02 23:08:50
7022,VJ Ld,,Transition to Commercial ev \nMust happen ASAP...,[],2021-12-02 23:34:37
7023,VJ Ld,,CLIMATE CHANGE RELATED TO ……\n\nWhich in turn ...,[],2021-12-02 23:36:40


In [ ]:
#tweets_df.to_csv('/drive/MyDrive/climatechange_tweets.csv')

In [ ]:
from datetime import datetime, date, timedelta

extract_datetime = datetime.today().strftime('%Y%m%d_%H%M%S')

filename = '/drive/MyDrive/climate_change_tweets_extracted_' + extract_datetime + '.csv'
tweets_df.to_csv(filename)

In [ ]:
filename

'/drive/MyDrive/climate_change_tweets_extracted_20211203_184229.csv'